# Idenfication of nearest ORFs

In [1]:
import os, re, sys
import pandas as pd, numpy as np
from matplotlib import pyplot as plt
from Bio import SeqIO
from scipy.stats import norm
from bisect import bisect_left
import seaborn as sns
sns.set_theme(style='whitegrid')

print(sys.version)

3.11.3 (main, May 15 2023, 10:43:03) [Clang 14.0.6 ]


## Parse genebank annotations

In [2]:
def parse_gb(gb):
    # get all sequence records for the specified genbank file
    recs = [rec for rec in SeqIO.parse(gb, "genbank")]
    # print the number of sequence records that were extracted
    # print(len(recs))
    # print annotations for each sequence record
    for rec in recs:
        annotation_type = rec.annotations
    # print the gene sequence feature summary information for each feature in each
    # sequence record
    dict = {}
    for rec in recs:
        feats = [feat for feat in rec.features if feat.type == "gene"]
        for feat in feats:
            locus_tag = feat.qualifiers['locus_tag']
            locus_tag = ";".join(locus_tag)
            location = feat.location
            strand = int(location.strand)
            # flip start and end if gene is on negative strand
            if strand == 1:
                start_pos = int(location.start)
                end_pos = int(location.end)
            elif strand == -1:
                start_pos = int(location.end)
                end_pos = int(location.start)
            dict.update({start_pos: (locus_tag, start_pos, end_pos, strand)})
    return dict

def make_orf_list(annotations, chrom):
    gb = os.path.join(annotations, chrom + '.gb')
    dict_chrom = parse_gb(gb)
    start_codon_list = sorted(list(dict_chrom.keys()))
    return dict_chrom, start_codon_list

annotations = "/Users/yunfei/2023_ChipSeq/annotations"
gene_dict = {}
start_codon_dict = {}
for fname, chrom in {'NZ_CP012004.1':'NZ_CP012004.1', 
                     'pAb1' : 'NC_009083.1', 
                     'pAb2' : 'NC_009084.1', 
                     'pAb3' : 'NZ_CP012005.1'}.items():
    dict_chrom, start_codon_list = make_orf_list(annotations, fname)
    gene_dict.update({chrom: dict_chrom})
    start_codon_dict.update({chrom : start_codon_list})


## Classify peak as coding or intergenic

In [3]:
def is_intra(summit, dict, chrom):
    # return whether a peak summit is within any CDS region
    match = False # intergenic
    for key, value in dict[chrom].items():
        start = value[1]
        end = value[2]
        chrom = value[3]
        if chrom == 1:
            if start <= summit <= end: # genes can overlap, thus there might be more than 1 hit
                match = True # coding
                break
        elif chrom == -1:
            if end <= summit <= start: # on the negative strand gene positions are [end, start]
                match = True # coding
                break
    return match

## Find the nearest ORFs
### Adjacent peaks on the same strand and within 1000 bp of the peak position are reported

In [47]:
class NearestORF:
    def __init__(self, summit, gene_dict, start_codon_dict, chrom):
        self.summit = summit
        self.chrom = chrom
        self.dict_annotation = gene_dict[chrom]
        self.start_codon_list = start_codon_dict[chrom]

    def find_nearest_start_codon(self):
        # find the start codon closest to the peak
        match_index = bisect_left(self.start_codon_list, self.summit)
        if match_index == 0:
            nearest_match = self.start_codon_list[0]
        elif match_index == len(self.start_codon_list):
            nearest_match = self.start_codon_list[-1]
        else:
            left = self.start_codon_list[match_index - 1]
            right = self.start_codon_list[match_index]
            if right - self.summit < self.summit - left:
                nearest_match = right
            else:
                nearest_match = left
        return match_index, nearest_match

    def get_match_info(self, match_ORF, strand):
        # for any matched ORF, get its corresponding gene annotation info (accession, gene coordinates, strand, distance to match)
        accession, start, end, ORF_strand = self.dict_annotation[match_ORF]
        if strand == 1:
            distance_to_match = self.summit - match_ORF
        elif strand == -1:
            distance_to_match = match_ORF - self.summit
        match_info = [self.summit, 0, accession, self.chrom, start, end, ORF_strand, distance_to_match]
        return match_info

    def find_next_ORFs(self, ORF_list_to_search, strand):
        # for a given match, search for the next nearest ORFs and output in a list
        # the search will be in one direction based on the strand info
        counter = 0
        next_ORFs = []
        for next_match_index in ORF_list_to_search:
            counter += 1
            next_match_strand = self.dict_annotation[next_match_index][3]
            if next_match_strand == strand:
                match_info = self.get_match_info(next_match_index, next_match_strand)
                match_info[1] = counter # Update Nth nearest ORF
                next_match_distance = match_info[7]
                if abs(next_match_distance) <= 1000:
                    next_ORFs.append(match_info)
                else:
                    break
            else:
                break
        return next_ORFs

    def nearest_orf(self):
        nearest_match_index, nearest_match = self.find_nearest_start_codon()
        nearest_match_info = self.get_match_info(nearest_match, self.dict_annotation[nearest_match][3])
        nearest_match_strand = self.dict_annotation[nearest_match][3]
        all_match = [nearest_match_info]
        print(nearest_match_index, nearest_match)

        if  nearest_match_strand == 1:  # If nearest ORF is on + strand, search right
            ORF_list = self.start_codon_list[(nearest_match_index + 1):]
        elif nearest_match_strand == -1:  # If nearest ORF is on - strand, search left
            ORF_list = self.start_codon_list[:nearest_match_index][::-1]  # List is reversed
        print(ORF_list)
        next_ORFs = self.find_next_ORFs(ORF_list, self.dict_annotation[nearest_match][3])

        all_match += next_ORFs
        return all_match

In [51]:
## To test NearestORF
summit = 306809
chrom = 'NZ_CP012004.1'
orf_finder = NearestORF(summit, gene_dict, start_codon_dict, chrom)
print(orf_finder.nearest_orf())
print('---------')
orf_finder2 = NearestORF(100260, gene_dict, start_codon_dict, chrom)
print(orf_finder2.nearest_orf())
print('---------')
print(NearestORF(1668799, gene_dict, start_codon_dict, chrom).nearest_orf())

292 306809
[[306809, 0, 'ACX60_RS01470', 'NZ_CP012004.1', 306809, 306509, -1, 0], [306809, 1, 'ACX60_RS01465', 'NZ_CP012004.1', 306393, 304884, -1, -416]]
---------
90 100165
[[100260, 0, 'ACX60_RS00450', 'NZ_CP012004.1', 100165, 99649, -1, -95], [100260, 1, 'ACX60_RS00450', 'NZ_CP012004.1', 100165, 99649, -1, -95], [100260, 2, 'ACX60_RS00445', 'NZ_CP012004.1', 99633, 98268, -1, -627]]
---------
1584 1668741
[[1668799, 0, 'ACX60_RS07835', 'NZ_CP012004.1', 1668741, 1668390, -1, -58], [1668799, 1, 'ACX60_RS07835', 'NZ_CP012004.1', 1668741, 1668390, -1, -58], [1668799, 2, 'ACX60_RS07830', 'NZ_CP012004.1', 1668373, 1668088, -1, -426], [1668799, 3, 'ACX60_RS07825', 'NZ_CP012004.1', 1668069, 1667757, -1, -730]]


In [61]:
test = start_codon_dict['NZ_CP012004.1']
print(test)
test.sort()
print(test)

[89, 1584, 2747, 3882, 6388, 7424, 9606, 9862, 11122, 12472, 13601, 14838, 16093, 16154, 18299, 19895, 20027, 20443, 23521, 23774, 25124, 25752, 27243, 28735, 29694, 30042, 32185, 33553, 35184, 36559, 39544, 40338, 40487, 41590, 44855, 45337, 46912, 47096, 47969, 49854, 50662, 50782, 52043, 52133, 52173, 54589, 55188, 55464, 56010, 57092, 57595, 58518, 58848, 59042, 59610, 60888, 63009, 63498, 64410, 65049, 65958, 66287, 66986, 68103, 69938, 70459, 72361, 73208, 73858, 75152, 75510, 77062, 77888, 80300, 80652, 81401, 82228, 83992, 85595, 87059, 88275, 89449, 91374, 92036, 93447, 94893, 95894, 98256, 99633, 100165, 101060, 101525, 101999, 103132, 104679, 104738, 106371, 106961, 107924, 108124, 109777, 110403, 111768, 112613, 113685, 116310, 117030, 117957, 118153, 120411, 121672, 123010, 123587, 124186, 124354, 125946, 126626, 131015, 131269, 133712, 134634, 135429, 136892, 137034, 138799, 139010, 139606, 142115, 143122, 143321, 145257, 146746, 147094, 147594, 148659, 148911, 149420, 14

In [48]:
def make_match_table(infile, outfile, gene_dict, start_codon_dict):
    # output ORF matches to a tsv file
    df_peak = pd.read_csv(infile, sep='\t')
    distance_list_intergenic = []
    distance_list_coding = []
    match_stats = []
    for index, peak in df_peak.iterrows():
        chrom = peak['chrom']
        summit = peak['average_summit']
        fold_enrich = peak['average_enrichment']
        orf_finder = NearestORF(summit, gene_dict, start_codon_dict, chrom)
        all_ORFs = orf_finder.nearest_orf()
        for match_info in all_ORFs:
            distance = match_info[-1]
            if is_intra(summit, gene_dict, chrom):
                match_info.append('coding')
                distance_list_coding.append(distance)
            else:
                match_info.append('intergenic')
                distance_list_intergenic.append(distance)
            match_info.append(fold_enrich)
            match_stats.append(match_info)
    
    # write output to csv
    # add 'Nth nearest ORF'
    col_names = ['summit_pos', 'Nth nearest ORF', 'locus_tag', 'chrom', 'start', 'end', 'strand', 'distance_to_match', 'match_type', 'average_fold_enrichment']
    df_out = pd.DataFrame(match_stats, columns=col_names)
    df_out = df_out.sort_values(by = ['summit_pos', 'Nth nearest ORF'], ascending = [True, True])
    print(df_out.head())
    df_out.to_csv(outfile, sep='\t', index=False)


## Write output as tsv file and plot the distribution of nearest ORFs

In [49]:
infile = "/Users/yunfei/2023_ChipSeq/average_peak_summit/BfmR-ChIP-49_seed1.average_summit.bed"
outdir = "/Users/yunfei/2023_ChipSeq/peak_stat_next_ORFs"

os.makedirs(outdir, exist_ok=True)

outfile = os.path.join(outdir, 'test_new.tsv')

make_match_table(infile, outfile, gene_dict, start_codon_dict)

10 10723
4 6196
3 3882
34 44855
38 47969
53 58848
58 63498
90 100165
96 104738
97 106961
106 116310
115 125946
125 138799
135 148911
141 155010
149 162457
154 168323
167 184851
170 188021
188 209237
193 218034
207 231073
242 260362
245 262183
246 263294
262 274131
268 279088
277 291602
287 301962
289 304519
291 304827
293 306809
295 308429
302 315955
304 319515
315 334925
325 344011
348 365619
357 374858
363 382518
366 384782
369 389424
385 405810
386 408606
393 417427
394 419366
395 419366
398 421269
404 427831
418 442233
420 442602
420 442602
432 448063
459 470220
462 473540
477 487263
479 490138
485 494650
503 514274
535 544306
555 564042
588 598102
617 634777
620 636586
622 640273
651 671635
657 678657
660 680964
665 687897
666 689988
672 695049
693 719020
697 721360
717 741762
732 756082
770 798379
773 802059
775 802389
776 803710
799 825591
805 832739
821 861138
824 862858
827 866656
834 869860
849 889413
852 893681
858 897531
923 968762
926 971844
976 1031861
979 1034491
991 104